In [7]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
## Python
import os
import random
import sys
import warnings
warnings.filterwarnings('ignore')


## Package
import glob 
import keras
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls
import seaborn as sns
import scipy.io.wavfile
import tensorflow as tf
py.init_notebook_mode(connected=True)


## Keras
from keras import regularizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

from keras.models import Model, Sequential
from keras import optimizers
from keras.layers import Input, Conv1D, Conv2D,BatchNormalization, MaxPooling1D,MaxPooling2D, LSTM, Dense, Activation, Layer,Reshape

from tensorflow.keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


## Sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


## Rest
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from tqdm import tqdm_notebook as tqdm

input_duration=3
# % pylab inline

In [9]:
# Data Directory
Ravdess_paths= np.array(("drive/MyDrive/dataset-dkn/Audio_Speech_Actors_01-24/","drive/MyDrive/dataset-dkn/Audio_Song_Actors_01-24/"))
dir_list = os.listdir(Ravdess_paths[0])
dir_list.sort()
print (dir_list)

[b'Actor_01', b'Actor_02', b'Actor_03', b'Actor_04', b'Actor_05', b'Actor_06', b'Actor_07', b'Actor_08', b'Actor_09', b'Actor_10', b'Actor_11', b'Actor_12', b'Actor_13', b'Actor_14', b'Actor_15', b'Actor_16', b'Actor_17', b'Actor_18', b'Actor_19', b'Actor_20', b'Actor_21', b'Actor_22', b'Actor_23', b'Actor_24']


In [10]:
# Create DataFrame for Data intel
ravdess_db = pd.DataFrame(columns=['path','source','actor', 'gender', 'emotion','emotion_lb'])
count = 0
for data_path in Ravdess_paths:
    dir_list = os.listdir(data_path)
    dir_list.sort()
    for i in dir_list:
        file_list = os.listdir(data_path + i.decode('utf-8'))
        for f in file_list:
            nm = f.split('.')[0].split('-')
            path = data_path + i.decode('utf-8') + '/' + f
            src = int(nm[1])
            actor = int(nm[-1])
            emotion = int(nm[2])
            source = "Ravdess"

            if int(actor)%2 == 0:
                gender = "female"
            else:
                gender = "male"

            if nm[3] == '01':
                intensity = 0
            else:
                intensity = 1

            if nm[4] == '01':
                statement = 0
            else:
                statement = 1

            if nm[5] == '01':
                repeat = 0
            else:
                repeat = 1

            if emotion == 1:
                lb = "neutral"
            elif emotion == 2:
                lb = "calm"
            elif emotion == 3:
                lb = "happy"
            elif emotion == 4:
                lb = "sad"
            elif emotion == 5:
                lb = "angry"
            elif emotion == 6:
                lb = "fearful"
            elif emotion == 7:
                lb = "disgust"
            elif emotion == 8:
                lb = "surprised"
            else:
                lb = "none"

            ravdess_db.loc[count] = [path,source,actor, gender, emotion,lb]
            count += 1

In [11]:
ravdess_db.head()

,path,source,actor,gender,emotion,emotion_lb
0,drive/MyDrive/dataset-dkn/Audio_Speech_Actors_...,Ravdess,1,male,1,neutral
1,drive/MyDrive/dataset-dkn/Audio_Speech_Actors_...,Ravdess,1,male,1,neutral
2,drive/MyDrive/dataset-dkn/Audio_Speech_Actors_...,Ravdess,1,male,2,calm
3,drive/MyDrive/dataset-dkn/Audio_Speech_Actors_...,Ravdess,1,male,2,calm
4,drive/MyDrive/dataset-dkn/Audio_Speech_Actors_...,Ravdess,1,male,1,neutral


In [12]:
ravdess_db.to_csv("drive/MyDrive/RAVDESS/model/list4_4a.csv")

In [13]:
ravdess_db['split'] =  np.where((ravdess_db.actor ==23) | (ravdess_db.actor ==24), 'Test', 
                                (np.where((ravdess_db.actor ==21) | (ravdess_db.actor ==22),'Val','Train')))

In [14]:
ravdess_db['split'].value_counts()

Train    2036
Val       208
Test      208
Name: split, dtype: int64

In [15]:
ravdess_db.drop(ravdess_db.index[ravdess_db['emotion_lb'] == 'surprised'], inplace = True)
ravdess_db.loc[ravdess_db.emotion_lb=='calm',['emotion','emotion_lb']]= 1,'neutral'

In [16]:
ravdess_db.emotion_lb.value_counts()

neutral    564
happy      376
sad        376
angry      376
fearful    376
disgust    192
Name: emotion_lb, dtype: int64

In [17]:
dataset_db = ravdess_db.copy()

In [18]:
dataset_db.emotion_lb = dataset_db.gender+"_"+dataset_db.emotion_lb

In [19]:
dataset_db.index=range(len(dataset_db.index))

In [20]:
dataset_db.to_csv("drive/MyDrive/RAVDESS/model/list_ML_clasic.csv")


In [21]:
dataset_db.emotion_lb.value_counts()
dataset_db.sort_values(by=['path'], inplace=True)
dataset_db.head()

,path,source,actor,gender,emotion,emotion_lb,split
1252,drive/MyDrive/dataset-dkn/Audio_Song_Actors_01...,Ravdess,1,male,1,male_neutral,Train
1249,drive/MyDrive/dataset-dkn/Audio_Song_Actors_01...,Ravdess,1,male,1,male_neutral,Train
1248,drive/MyDrive/dataset-dkn/Audio_Song_Actors_01...,Ravdess,1,male,1,male_neutral,Train
1251,drive/MyDrive/dataset-dkn/Audio_Song_Actors_01...,Ravdess,1,male,1,male_neutral,Train
1258,drive/MyDrive/dataset-dkn/Audio_Song_Actors_01...,Ravdess,1,male,1,male_neutral,Train


In [22]:
dataset_db.index = range(len(dataset_db.index))

In [23]:
dataset_db.shape

(2260, 7)

In [24]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [26]:
import h5py
with h5py.File('drive/MyDrive/RAVDESS/model/Ravdess_audio_Mel_spec.h5', 'r') as hf:
  audios = hf['Ravdess_audio_Mel_spec'][:]

In [27]:
x_train = audios[(dataset_db['split'] == 'Train')]
y_train = dataset_db.emotion_lb[(dataset_db['split'] == 'Train')]

print(x_train.shape,y_train.shape)

(1876, 128, 259, 1) (1876,)


In [29]:
x_test = audios[(dataset_db['split'] == 'Val')]
y_test = dataset_db.emotion_lb[(dataset_db['split'] == 'Val')]

In [30]:
X_train = np.array(x_train)
y_train = np.array(y_train)
X_test = np.array(x_test)
y_test = np.array(y_test)

In [36]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2]*X_train.shape[3])

In [37]:
X_train.shape

(1876, 33152)

In [39]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2]*X_test.shape[3])

In [40]:
X_test.shape

(192, 33152)

In [41]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

0.21875
                precision    recall  f1-score   support

  female_angry       0.00      0.00      0.00        16
female_disgust       0.11      0.12      0.12         8
female_fearful       0.00      0.00      0.00        16
  female_happy       0.00      0.00      0.00        16
female_neutral       0.41      0.58      0.48        24
    female_sad       0.00      0.00      0.00        16
    male_angry       0.12      1.00      0.22        16
  male_disgust       0.00      0.00      0.00         8
  male_fearful       0.00      0.00      0.00        16
    male_happy       0.00      0.00      0.00        16
  male_neutral       0.65      0.46      0.54        24
      male_sad       0.00      0.00      0.00        16

      accuracy                           0.22       192
     macro avg       0.11      0.18      0.11       192
  weighted avg       0.15      0.22      0.15       192

[[ 0  1  0  0  1  0 14  0  0  0  0  0]
 [ 0  1  0  0  3  0  4  0  0  0  0  0]
 [ 0  0  0  0  

In [50]:
from sklearn.metrics import f1_score
f1_score(y_true=y_test,y_pred=dtree_predictions, average='macro') *100

28.367826019999935

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test,y_pred=dtree_predictions) *100

31.770833333333332

In [51]:
from sklearn.metrics import recall_score
recall_score(y_true=y_test,y_pred=dtree_predictions, average='macro') *100

30.208333333333332

In [54]:
from sklearn.metrics import precision_score
precision_score(y_true=y_test,y_pred=dtree_predictions, average='macro') *100

35.14258546227856

In [42]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 6).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

0.3177083333333333
                precision    recall  f1-score   support

  female_angry       0.43      0.38      0.40        16
female_disgust       1.00      0.12      0.22         8
female_fearful       0.30      0.38      0.33        16
  female_happy       0.20      0.38      0.26        16
female_neutral       0.46      0.50      0.48        24
    female_sad       0.71      0.31      0.43        16
    male_angry       0.48      0.69      0.56        16
  male_disgust       0.00      0.00      0.00         8
  male_fearful       0.14      0.19      0.16        16
    male_happy       0.21      0.44      0.28        16
  male_neutral       0.00      0.00      0.00        24
      male_sad       0.29      0.25      0.27        16

      accuracy                           0.32       192
     macro avg       0.35      0.30      0.28       192
  weighted avg       0.33      0.32      0.29       192

[[ 6  0  0  2  1  0  1  0  5  1  0  0]
 [ 0  1  3  1  2  0  0  0  0  0  0  1]
 [ 4

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test,y_pred=dtree_predictions) *100

31.770833333333332

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_true=y_test,y_pred=dtree_predictions, average='macro') *100

30.208333333333332

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_true=y_test,y_pred=dtree_predictions, average='macro') *100

35.14258546227856

In [43]:
from sklearn.svm import SVC 
svm_model_linear = SVC().fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, svm_predictions) )

0.5
                precision    recall  f1-score   support

  female_angry       0.48      0.62      0.54        16
female_disgust       0.50      0.50      0.50         8
female_fearful       0.71      0.31      0.43        16
  female_happy       0.47      0.44      0.45        16
female_neutral       0.57      0.96      0.72        24
    female_sad       0.60      0.38      0.46        16
    male_angry       0.46      0.75      0.57        16
  male_disgust       0.00      0.00      0.00         8
  male_fearful       0.22      0.12      0.16        16
    male_happy       0.43      0.75      0.55        16
  male_neutral       0.60      0.50      0.55        24
      male_sad       1.00      0.19      0.32        16

      accuracy                           0.50       192
     macro avg       0.50      0.46      0.44       192
  weighted avg       0.53      0.50      0.47       192

[[10  2  0  3  1  0  0  0  0  0  0  0]
 [ 2  4  0  2  0  0  0  0  0  0  0  0]
 [ 3  0  5  3  2  3

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test,y_pred=svm_predictions) *100

60.9375

In [56]:
from sklearn.metrics import recall_score
recall_score(y_true=y_test,y_pred=svm_predictions, average='macro') *100

55.90277777777778

In [57]:
from sklearn.metrics import precision_score
precision_score(y_true=y_test,y_pred=svm_predictions, average='macro') *100

61.03291055922634

In [44]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, svm_predictions) )

0.609375
                precision    recall  f1-score   support

  female_angry       0.61      0.69      0.65        16
female_disgust       0.50      0.38      0.43         8
female_fearful       0.80      0.25      0.38        16
  female_happy       0.91      0.62      0.74        16
female_neutral       0.69      1.00      0.81        24
    female_sad       0.53      0.62      0.57        16
    male_angry       0.39      0.81      0.53        16
  male_disgust       0.00      0.00      0.00         8
  male_fearful       0.60      0.38      0.46        16
    male_happy       0.52      0.81      0.63        16
  male_neutral       0.78      0.58      0.67        24
      male_sad       1.00      0.56      0.72        16

      accuracy                           0.61       192
     macro avg       0.61      0.56      0.55       192
  weighted avg       0.65      0.61      0.59       192

[[11  1  1  0  3  0  0  0  0  0  0  0]
 [ 3  3  0  0  0  1  1  0  0  0  0  0]
 [ 4  0  4  1 

In [45]:
from sklearn.ensemble import RandomForestClassifier
  
 # create regressor object 
classifier = RandomForestClassifier(n_estimators = 100, random_state = 0) 
  
# fit the regressor with x and y data 
classifier.fit(X_train, y_train)   

c_p = classifier.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=c_p))
print(classification_report(y_test,c_p)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,c_p) )

0.5104166666666666
                precision    recall  f1-score   support

  female_angry       0.56      0.56      0.56        16
female_disgust       0.57      0.50      0.53         8
female_fearful       0.56      0.31      0.40        16
  female_happy       0.64      0.56      0.60        16
female_neutral       0.70      0.88      0.78        24
    female_sad       0.56      0.56      0.56        16
    male_angry       0.36      0.88      0.51        16
  male_disgust       0.60      0.38      0.46         8
  male_fearful       0.00      0.00      0.00        16
    male_happy       0.36      0.62      0.45        16
  male_neutral       0.72      0.54      0.62        24
      male_sad       0.25      0.06      0.10        16

      accuracy                           0.51       192
     macro avg       0.49      0.49      0.47       192
  weighted avg       0.50      0.51      0.48       192

[[ 9  2  2  1  0  0  0  0  0  2  0  0]
 [ 1  4  0  2  0  0  0  0  0  1  0  0]
 [ 6

In [58]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test,y_pred=c_p) *100

51.041666666666664

In [59]:
from sklearn.metrics import recall_score
recall_score(y_true=y_test,y_pred=c_p, average='macro') *100

48.78472222222223

In [60]:
from sklearn.metrics import precision_score
precision_score(y_true=y_test,y_pred=c_p, average='macro') *100

49.0265059015059